### 내용 기반 필터링 추천

> 각 아이템 간의 유사도 측정

> 추천 대상이 되는 사용자가 선호하는 아이템을 선정한다

> 위에서 선정된 아이템과 가장 유사도가 높은 N개의 아이템을 찾는다.

> 이 N개의 아이템을 사용자에게 추천한다

In [1]:
import pandas as pd

movies = pd.read_csv('/content/drive/MyDrive/recosys/movies_metadata.csv', encoding = 'latin-1', low_memory = False)
movies = movies[['id', 'title', 'overview']]
movies.head(10)

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...
5,949,Heat,"Obsessive master thief, Neil McCauley leads a ..."
6,45325,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses..."
7,9091,Sudden Death,International action superstar Jean Claude Van...
8,710,GoldenEye,James Bond must unmask the mysterious head of ...
9,9087,The American President,"Widowed U.S. president Andrew Shepherd, one of..."


In [2]:
#데이터 전처리
movies['overview'] = movies['overview'].fillna(' ')
movies = movies.dropna()
len(movies)

#불용어를 'English'로 지정하고 tf-idf 계산
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

#Cosine 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index = movies.index, columns = movies.index)

In [4]:
indices = pd.Series(movies.index, index = movies['title'])
'''Series로 바꾸는 이유는 함수 실행 시 영화 인덱스 값(Value)를 빨리 찾기 위함.
Series로 바꾸면 영화 제목을 'Key'로 갖고, 영화 인덱스를 'Value'로 갖는 형태가 됨'''


def content_recommender(title, n_of_recomm):
  idx = indices[title]

  sim_scores = cosine_sim[idx]

  sim_scores = sim_scores.sort_values(ascending = False)[1:n_of_recomm + 1]

  return movies.loc[sim_scores.index]['title']

print(content_recommender('The Lion King', 5))   #The Lion King과 유사한 영화 5개
print(content_recommender('The Dark Knight Rises', 10))   #The Dark Knight Rises와 유사한 영화 10개

34664    How the Lion Cub and the Turtle Sang a Song
9339                               The Lion King 1Â½
9101                  The Lion King 2: Simba's Pride
42806                                           Prey
25637                                 Fearless Fagan
Name: title, dtype: object
12468                                      The Dark Knight
149                                         Batman Forever
1321                                        Batman Returns
15497                           Batman: Under the Red Hood
584                                                 Batman
21179    Batman Unmasked: The Psychology of the Dark Kn...
9216                    Batman Beyond: Return of the Joker
18021                                     Batman: Year One
19778              Batman: The Dark Knight Returns, Part 1
3085                          Batman: Mask of the Phantasm
Name: title, dtype: object
